In [1]:
import drms
import json, numpy as np, matplotlib.pylab as plt, matplotlib.ticker as mtick
import urllib
from astropy.io import fits
from skimage.transform import resize
from sunpy.visualization.colormaps import color_tables as ct

In [5]:
import drms
c = drms.Client()
c.series(r'hmi\.sharp_')
# Set a series
si = c.info('hmi.sharp_cea_720s')

#### You only need to change the date and path in the name in the cells below

In [11]:
#example for how to query
#This queries all the data from 2021.09.01 - 2022.01.01 where the unsigned magnetic flux > 4e21 and CRLN_OBS < 360 and CRL_OBS < 360
#The property that are queried are indicated at the end as the key argument, so you can obviously add more properties there

date = '2021.09.01 - 2022.01.01'    #change this to your desired time period
keys = c.query('hmi.sharp_cea_720s[][{}][? (CRLN_OBS < 360) AND (CRLT_OBS < 360) AND (USFLUX > 4e21) ?]'.format(date), key='T_REC, HARPNUM, USFLUX, CRLT_OBS, CRLN_OBS, AREA_ACR')


In [6]:
#just a bunch of timespans you want to loop through
#they have lengths of 3 months because if you make them too long your query time gets killed

date = ['2021.09.01 - 2022.01.01', '2021.06.01 - 2021.09.01', '2021.03.01 - 2021.06.01', '2021.01.01 - 2021.03.01']
date2 = ['2020.09.01 - 2021.01.01', '2020.06.01 - 2020.09.01', '2020.03.01 - 2020.06.01', '2020.01.01 - 2020.03.01']
date3 = ['2019.09.01 - 2020.01.01', '2019.06.01 - 2019.09.01', '2019.03.01 - 2019.06.01', '2019.01.01 - 2019.03.01']
date4 = ['2018.09.01 - 2019.01.01', '2018.06.01 - 2018.09.01', '2018.03.01 - 2018.06.01', '2018.01.01 - 2018.03.01']
date5 = ['2017.09.01 - 2018.01.01', '2017.06.01 - 2017.09.01', '2017.03.01 - 2017.06.01', '2017.01.01 - 2017.03.01']
date6 = ['2016.09.01 - 2017.01.01', '2016.06.01 - 2016.09.01', '2016.03.01 - 2016.06.01', '2016.01.01 - 2016.03.01']
date7 = ['2015.09.01 - 2016.01.01', '2015.06.01 - 2015.09.01', '2015.03.01 - 2015.06.01', '2015.01.01 - 2015.03.01']
date8 = ['2014.09.01 - 2015.01.01', '2014.06.01 - 2014.09.01', '2014.03.01 - 2014.06.01', '2014.01.01 - 2014.03.01']
date9 = ['2013.09.01 - 2014.01.01', '2013.06.01 - 2013.09.01', '2013.03.01 - 2013.06.01', '2013.01.01 - 2013.03.01']
date10 = ['2012.09.01 - 2013.01.01', '2012.06.01 - 2012.09.01', '2012.03.01 - 2012.06.01', '2012.01.01 - 2012.03.01']
date11 = ['2011.09.01 - 2012.01.01', '2011.06.01 - 2011.09.01', '2011.03.01 - 2011.06.01', '2011.01.01 - 2011.03.01']
date12 = ['2010.09.01 - 2011.01.01', '2010.06.01 - 2010.09.01', '2010.03.01 - 2010.06.01', '2010.01.01 - 2010.03.01']
date = date + date2 + date3 + date4 + date5 + date6 + date7 + date8 + date9 + date10 + date11 + date12

date.reverse()

In [7]:
for d in date:
    try:
        keys = c.query('hmi.sharp_cea_720s[][{}][? (CRLN_OBS < 360) AND (CRLT_OBS < 360) AND (USFLUX > 4e21) ?]'.format(d), key='T_REC, HARPNUM, USFLUX, CRLT_OBS, CRLN_OBS, AREA_ACR') #at the end here to the key argument you want to add additional properties
        keys.to_csv(d+'.csv', index = 'False')
        #print(d)
    except:
        print('Failed for:', d)
        #it fails if there was too much data
        pass
        

Failed for: 2011.09.01 - 2012.01.01
Failed for: 2014.09.01 - 2015.01.01
